<a href="https://colab.research.google.com/github/naashonomics/projects_python/blob/main/groq_LLaVA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install groq
!pip install gradio

In [ ]:
from getpass import getpass
import os

# Securely input your API key
api_key = getpass("Enter your API key: ")

# Set it as an environment variable
os.environ["GROQ_API_KEY"] = api_key



In [ ]:

from groq import Groq

client = Groq()
completion = client.chat.completions.create(
    model="llava-v1.5-7b-4096-preview",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What's in this image?"},
                {"type": "image_url", "image_url": {"url": "https://images.pexels.com/photos/3225517/pexels-photo-3225517.jpeg"}}
            ]
        },
    ],
    temperature=1,
    max_tokens=1024,
    top_p=1,
    stream=False,
    stop=None,
)

print(completion.choices[0].message.content)

The image features a beautiful landscape with dried-up water running through a canyon in a rocky gorge. Trees in the scene are in varying positions, some close to the water and others further away, creating a diverse view of the canyon. Numerous clouds in the sky add an atmospheric touch to the scene. This picturesque place seems to be situated in a desert-like environment, as there is a sinking branch sitting in the water of the creek. Overall, the image exudes a sense of tranquility and natural beauty.


In [ ]:
import gradio as gr
from groq import Groq
import time

# Function to analyze an image from a URL with retry mechanism
def analyze_image(image_url, retries=3, delay=2):
    for attempt in range(retries):
        try:
            client = Groq()
            completion = client.chat.completions.create(
                model="llava-v1.5-7b-4096-preview",
                messages=[
                    {
                        "role": "user",
                        "content": [
                            {"type": "text", "text": "What's in this image?"},
                            {"type": "image_url", "image_url": {"url": image_url}}
                        ]
                    },
                ],
                temperature=1,
                max_tokens=1024,
                top_p=1,
                stream=False,
                stop=None,
            )
            return completion.choices[0].message.content
        except Exception as e:
            if attempt < retries - 1:
                print(f"API issue encountered: {e}. Retrying in {delay} seconds...")
                time.sleep(delay)  # Wait before retrying
            else:
                return f"Failed after {retries} attempts. Error: {e}"

# Gradio interface
def gradio_interface(image_url):
    if image_url:
        # Analyze the image and return both the image and the AI's response
        ai_response = analyze_image(image_url)
        return image_url, ai_response
    else:
        return None, "Please provide an image URL."

# Create a Gradio interface with both image and text outputs
iface = gr.Interface(
    fn=gradio_interface,
    inputs=gr.Textbox(label="Enter Image URL"),
    outputs=[gr.Image(label="Image"), gr.Textbox(label="AI's Response")],
    title="Image Analyzer with AI",
    description="Enter an image URL to see what the AI thinks is in the image."
)

# Launch the Gradio interface
iface.launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://e67b325995d945bb6e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
